In [ ]:
# first install python 3.6
!sudo apt-get update -y
!sudo apt-get install python3.6
# change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.6 1
# select python version
!sudo update-alternatives --config python3
# check python version
!python --version
# install pip for new python 
!sudo apt-get install python3.6-distutils
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py
# upgrade pip
!sudo apt install python3-pip
!python -m pip install --upgrade pip

In [2]:
X_sentence=list()
Y_sentence=list()
count=0
base_path="/home/lorenzobgl/projects/NER_Manufacturing-FabNER"
X_train=list()
Y_train=list()
X_val=list()
Y_val=list()
X_test=list()
Y_test=list()
with open(base_path+"/S1-test.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_test.append(X_sentence)      
                Y_test.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
with open(base_path+"/S2-train.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_train.append(X_sentence)      
                Y_train.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
with open(base_path+"/S3-val.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_val.append(X_sentence)      
                Y_val.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
# sentences it is a list of sentences. Every sentence is a list of tuple
tokenized_sentences=X_train+X_test+X_val
tokenized_labels=Y_train+Y_test+Y_val

In [ ]:
tokenized_sentences_modified=[]
technical_word=""
for sent in tokenized_sentences:
    tokenized_sent_modified=[]
    for el in sent:
        if el[1][0]=="O" or el[1][0]=="S":
            tokenized_sent_modified.append(el[0])
        else:
            if el[1][0]=="B" or el[1][0]=="I":
                technical_word=technical_word+el[0]+"_"
            if el[1][0]=="E":
                tokenized_sent_modified.append(technical_word+el[0])
                technical_word=""
    tokenized_sentences_modified.append(tokenized_sent_modified)

In [3]:
from gensim.models import FastText

# model_1=FastText(sentences=tokenized_sentences_modified, vector_size=300, window=10, min_count=3, sg=1, negative=10, alpha=0.01, sample=0.0001)
model_2=FastText(sentences=tokenized_sentences, vector_size=100, window=10, min_count=5, sg=1, negative=10, alpha=0.01, sample=0.0001)

In [4]:
# model_1.train(tokenized_sentences_modified, total_examples=len(tokenized_sentences_modified), epochs=20)
model_2.train(tokenized_sentences, total_examples=len(tokenized_sentences), epochs=20)

(3047633, 6991040)

In [ ]:
# model_3=Word2Vec(sentences=tokenized_sentences_modified, vector_size=300, window=10, min_count=5, sg=1, negative=10, alpha=0.01, sample=0.0001)
# model_3.train(tokenized_sentences_modified, total_examples=len(tokenized_sentences_modified), epochs=20)

In [7]:
import importlib
from modelli import wrapper, utils
importlib.reload(wrapper)
from modelli.wrapper import Sequence


lstm=Sequence(embeddings=model_2.wv, initial_vocab=model_2.wv.key_to_index)
lstm.fit_vocab(X_train,Y_train)

In [ ]:
lstm.p._word_vocab.vocab

In [ ]:
from modelli.wrapper import Sequence
import pickle

lstm=Sequence.load("param","weights.h5","preprocessor")
lstm.model.compile( optimizer=lstm.optimizer)
with open('optimizer.pkl', 'rb') as f:
    weight_values = pickle.load(f)
lstm.model.optimizer.set_weights(weight_values)


In [8]:
from modelli.trainer import Trainer

trainer = Trainer(lstm.model, preprocessor=lstm.p)
trainer.train(X_train, Y_train, X_val, Y_val,epochs=1, batch_size=32,verbose=1,shuffle=True)


Epoch 1/1
131/295 [============>.................] - ETA: 2:14 - loss: 8.1768

In [8]:
from keras.backend import batch_get_value
import pickle

# lstm.save("param","weights.h5","preprocessor")
symbolic_weights = getattr(lstm.model.optimizer, 'weights')
weight_values = batch_get_value(symbolic_weights)
with open('optimizer.pkl', 'wb') as f:
    pickle.dump(weight_values, f)


In [ ]:
from modelli.wrapper import Sequence

lstm=Sequence (word_embedding_dim=100, embeddings=model_2.wv)
lstm.fit(X_train, Y_train, X_val, Y_val, epochs=1)

In [ ]:
from modelli.models import save_model

save_model(lstm.model, "weights.h5","param")

In [ ]:
lstm.fit(X_train, Y_train, X_val, Y_val,epochs=10)